<a href="https://colab.research.google.com/github/md2383/SeminarInAI/blob/main/Week06_bert/homework_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework 5 (10pt): Question search engine

Remeber Week01, where you used GloVe embeddings to find related questions? That was... cute. Now, it's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `practice06.ipynb` [notebook](https://github.com/anton-selitskiy/RIT_LLM/blob/main/Week06_bert/practice06.ipynb)

This assignmend is inspired by this [notebook](https://github.com/yandexdataschool/nlp_course/blob/2024/week05_transfer/homework.ipynb)

In [ ]:
#%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

In [ ]:
print(tokenizer.decode(qqp_preprocessed['train'][0]['input_ids']))

### Task 1: evaluation (3 point)

We randomly chose a model trained on QQP - but is it any good?

One way to assess this is by measuring validation accuracy, which you will implement next.

Here’s the interface to help you get started:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=2
)

In [ ]:
for batch in val_loader:
    break  # here will be your training code later. For now, it reads one batch
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
# <A lot of YOUR CODE HERE>
# ...


accuracy = #<Validation accuracy, between 0 and 1>


In [ ]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (5 points)

Fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base), but you can choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually (as we did in class) or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.

In [ ]:
# model_name = "microsoft/deberta-v3-base"
# tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
# model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
<A whole lot of your code goes here>

### Task 3: try the full pipeline (2 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 3 examples.